In [3]:
from pyspark import SparkContext # Antiguo
from pyspark.sql import SparkSession # Nuevo

In [4]:
spark = SparkSession.builder.master("local").appName("PrimerSession").getOrCreate()

In [5]:
spark.stop()# terminar session NECESARIO

In [6]:
sc = SparkContext(master="local", appName="PrimerContexto")

In [7]:
rdd = sc.parallelize([1,2,3])

In [8]:
type(rdd)

pyspark.rdd.RDD

In [9]:
rdd.collect()

[1, 2, 3]

In [10]:
sc

<SparkContext master=local appName=PrimerContexto>

In [11]:
!ls /app/curso-apache-spark-platzi/files

deporte.csv	 deportistaError.csv  modelo_relacional.jpg
deportista2.csv  evento.csv	      paises.csv
deportista.csv	 juegos.csv	      resultados.csv


In [12]:
path = "/app/curso-apache-spark-platzi/files/"

equipos = sc.textFile(path+"paises.csv").map(lambda line: line.split(","))
equipos.take(5) # head

[['id', 'equipo', 'sigla'],
 ['1', '30. Februar', 'AUT'],
 ['2', 'A North American Team', 'MEX'],
 ['3', 'Acipactli', 'MEX'],
 ['4', 'Acturus', 'ARG']]

In [13]:
equipos.map(lambda x: (x[2])).distinct().count()

231

In [14]:
# agrupa por paises
equipos.map(lambda x: (x[2],x[1])).groupByKey().mapValues(len).take(5)

[('sigla', 1), ('AUT', 11), ('MEX', 9), ('ARG', 18), ('AFG', 1)]

In [15]:
argentina = equipos.filter(lambda x: "ARG" in x)
argentina.take(5)

[['4', 'Acturus', 'ARG'],
 ['37', 'Antares', 'ARG'],
 ['42', 'Arcturus', 'ARG'],
 ['43', 'Ardilla', 'ARG'],
 ['45', 'Argentina', 'ARG']]

In [16]:
argentina.count()

18

In [17]:
# cuenta en milisegundos 20 miliseg
argentina.countApprox(20)

18

In [18]:
# csv union  
deportista_1 = sc.textFile(path+"deportista.csv").map(lambda x: x.split(","))
deportista_2 = sc.textFile(path+"deportista2.csv").map(lambda x: x.split(","))

deportistas_unidos = deportista_1.union(deportista_2)

In [19]:
deportistas_unidos.count()

135572

In [20]:
deportistas_unidos.take(3)

[['deportista_id', 'nombre', 'genero', 'edad', 'altura', 'peso', 'equipo_id'],
 ['1', 'A Dijiang', '1', '24', '180', '80', '199'],
 ['2', 'A Lamusi', '1', '23', '170', '60', '199']]

In [21]:
deportistas_paises = deportistas_unidos.map(lambda x: [x[-1],x[:-1]]).join(equipos.map(lambda x: [x[0],x[2]]))
deportistas_paises.take(3)

[('199', (['1', 'A Dijiang', '1', '24', '180', '80'], 'CHN')),
 ('199', (['2', 'A Lamusi', '1', '23', '170', '60'], 'CHN')),
 ('199', (['602', 'Abudoureheman', '1', '22', '182', '75'], 'CHN'))]

In [22]:
# read resultados
resultado = sc.textFile(path+"resultados.csv").map(lambda x: x.split(","))
resultado.take(3)

[['resultado_id', 'medalla', 'deportista_id', 'juego_id', 'evento_id'],
 ['1', 'NA', '1', '39', '1'],
 ['2', 'NA', '2', '49', '2']]

In [23]:
# seleccionar campeones
ganadores = resultado.filter(lambda x: "NA" not in x[1])
ganadores.take(3)

[['resultado_id', 'medalla', 'deportista_id', 'juego_id', 'evento_id'],
 ['4', 'Gold', '4', '2', '4'],
 ['38', 'Bronze', '15', '7', '19']]

In [27]:
# ganadores por paises
todo = deportistas_paises.join(ganadores)
todo.take(3)

[('1090',
  ((['9150', 'Yelena Yuryevna Bechke Petrova Ellis ', '2', '26', '158', '48'],
    'EUN'),
   'Bronze')),
 ('1090',
  ((['9819', 'Gennady Vladimirovich Belyakov', '1', '23', '171', '84'], 'EUN'),
   'Bronze')),
 ('1090',
  ((['41896', 'Andrey Vladimirovich Gorokhov', '1', '23', '185', '92'], 'EUN'),
   'Bronze'))]

In [29]:
valores_medallas = {"Gold": 7, "Silver":5, "Bronze": 4}

In [35]:
valores_paises = todo.map(lambda x: (x[1][0][-1], valores_medallas[x[1][1]]) )
valores_paises.take(3)

[('EUN', 4), ('EUN', 4), ('EUN', 4)]

In [37]:
from operator import add

In [41]:
concluido = valores_paises.reduceByKey((add)).sortBy(lambda x: x[1], ascending=False)
concluido.take(3)                                                                    

[('CAN', 32538), ('ARG', 12520), ('HUN', 10860)]

In [55]:
sc.stop()

# Dataframe (Catalyst)

In [51]:
from pyspark import SparkContext # Antiguo
from pyspark.sql import SparkSession # Nuevo

from pyspark.sql.types import StructType, StructField, StringType, FloatType, IntegerType
from pyspark.sql.types import Row
from pyspark.sql import SQLContext

In [56]:
sc = SparkSession.builder.appName("DF").getOrCreate()
scsql = SQLContext(sc)

In [60]:
path = "./curso-apache-spark-platzi/files/"

In [61]:
juego = StructType([
    StructField("juego_id", IntegerType(), False),
    StructField("year", StringType(),False),
    StructField("temporada",StringType(), False),
    StructField("ciudad",StringType(), False),
])

juegoDF = scsql.read.schema(juego).option("header", "true").csv(path+"juegos.csv")

In [62]:
juegoDF.show(4)

+--------+-----------+---------+------+
|juego_id|       year|temporada|ciudad|
+--------+-----------+---------+------+
|       1|1896 Verano|     1896|Verano|
|       2|1900 Verano|     1900|Verano|
|       3|1904 Verano|     1904|Verano|
|       4|1906 Verano|     1906|Verano|
+--------+-----------+---------+------+
only showing top 4 rows



In [64]:
sc

In [3]:
lista = [1,2,3,4,5]

In [4]:
rdd  = sc.parallelize(lista)

In [10]:
rdd.collect()

[1, 2, 3, 4, 5]

# Filtrado

In [9]:
filtrado = rdd.filter(lambda x : x>1)
filtrado.collect()

[2, 3, 4, 5]

# map

In [20]:
def suma(x): return (x+1)

suma_rdd = rdd.map(suma)
suma_rdd.collect()

[2, 3, 4, 5, 6]

In [32]:
cuadrado = rdd.map(suma).map(lambda x: (x,x**2))

cuadrado.collect()

[(2, 4), (3, 9), (4, 16), (5, 25), (6, 36)]

# flatmap lista simple

In [31]:
flatmap = rdd.map(suma).flatMap(lambda x: (x,x**2))

flatmap.collect() # convierte a una lista simple

[2, 4, 3, 9, 4, 16, 5, 25, 6, 36]

# duplas

In [34]:
duplas = [("a",1), ("b",2), ("c",3)]
duplas


[('a', 1), ('b', 2), ('c', 3)]

In [35]:
duplas_rdd = sc.parallelize(duplas)
duplas_rdd.collect()

[('a', 1), ('b', 2), ('c', 3)]

# duplas con zip

In [39]:
zip_rdd = sc.parallelize(zip((["a","b","c"]), range(1,4,1)))

zip_rdd.collect()

[('a', 1), ('b', 2), ('c', 3)]

In [40]:
# persitencia y cached

In [43]:
rdd.is_cached

False

In [46]:
rdd.cache()

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:287

In [47]:
rdd.is_cached

True

In [48]:
rdd.getStorageLevel() # diferentes formas de almacenamiento

StorageLevel(False, True, False, False, 1)

In [50]:
print(rdd.getStorageLevel())

Memory Serialized 1x Replicated


In [51]:
rdd_2 = rdd.map(lambda x: x+1)

In [52]:
from pyspark import StorageLevel

In [54]:
rdd_2.persist(StorageLevel.MEMORY_AND_DISK_2)

PythonRDD[16] at RDD at PythonRDD.scala:53

In [55]:
rdd_2.getStorageLevel()

StorageLevel(True, True, False, False, 2)

In [56]:
print(rdd_2.getStorageLevel())

Disk Memory Serialized 2x Replicated


# la persistencia no se hereda

In [58]:
rdd_3 = rdd_2.map(lambda x: x+2)

In [60]:
rdd_2.is_cached

True

In [62]:
rdd_3.is_cached

False

In [63]:
# particionado

In [64]:
# Obtiene el N de particiones
rdd.getNumPartitions()

12

In [65]:
rdd.collect()

[1, 2, 3, 4, 5]

In [68]:
# Como dividio las particiones
rdd.glom().collect()

[[], [], [1], [], [2], [], [], [3], [], [4], [], [5]]

In [71]:
# El particionado se Hereda
rdd_2.glom().collect()

[[], [], [2], [], [3], [], [], [4], [], [5], [], [6]]

In [75]:
rdd_22 = rdd_2.repartition(2)

In [76]:
rdd_22.getNumPartitions()

2

In [77]:
rdd_22.glom().collect()

[[2, 3, 5, 6], [4]]

In [78]:
# Spark sql dataframe

In [80]:
from pyspark.sql import SparkSession

In [81]:
spark = SparkSession.builder.appName("DataFrame").getOrCreate()

In [ ]:
df = spark.read.csv("arcvio.csv")


In [ ]:
df.printSchema()

In [ ]:
df.columns

In [ ]:
df.describe().show()

In [82]:
# Schema

In [83]:
from pyspark.sql.types import StringType, StructField, StringType, IntegerType

In [ ]:
schema = [StructField]